In [2]:
!pip install h5py

In [3]:
from models.DcaseNet import get_DcaseNet_v3
from utils import * 
import torch 
import os
import torch.nn as nn 
import torch.nn.functional as F
import pickle as pk
from dataloaders import get_loaders_ASC, get_loaders_SED, get_loaders_TAG

# from parser import get_args

# args = get_args()
cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')
PATH = '/DATA/G3/Datasets/archive/Original_split/TAU-urban-acoustic-scenes-2020-mobile-development'
evaluation_setup = 'evaluation_setup'
wav_file = 'audio'
meta_scp = os.path.join(PATH  , 'meta.csv')
d_label_ASC_pth = os.path.join(PATH , evaluation_setup , 'd_label.pkl')
fold_trn = os.path.join(PATH , evaluation_setup , 'fold1_train.csv')
fold_evl = os.path.join(PATH , evaluation_setup , 'fold1_evaluate.csv')
verbose = 1 
d_label_ASC = [] 
l_label_ASC = []
# if args.reproducible:
#         np.random.seed(args.seed)
#         torch.manual_seed(args.seed)
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = False

### For the ASC Dcase Model pre-trained ###
lines_ASC = get_utt_list(os.path.join(PATH , wav_file))

# try :

if os.path.exists(d_label_ASC_pth):
    d_label_ASC, l_label_ASC = pk.load(open(d_label_ASC_pth, 'rb'))
    print('Pickle File Exists')
# except FileNotFoundError: 
else :
    with open(meta_scp , 'r') as f:
        l_meta_ASC = f.readlines()
    d_label_ASC, l_label_ASC = make_d_label(l_meta_ASC[1:])
    pk.dump([d_label_ASC, l_label_ASC], open(d_label_ASC_pth, 'wb'))
    print('Pickle File Dumped')

trn_lines_ASC = split_dcase2020_fold_strict(fold_scp = fold_trn, lines = lines_ASC)
evl_lines_ASC = split_dcase2020_fold_strict(fold_scp = fold_evl, lines = lines_ASC)
if verbose > 0 :
    print('ASC DB statistics')
    print('# trn samp: {}\n# evl samp: {}'.format(len(trn_lines_ASC), len(evl_lines_ASC)))
    print(d_label_ASC)
    print(l_label_ASC)

DcaseNet_v3 = get_DcaseNet_v3()
model =  DcaseNet_v3
pre_trained_model = ('./weights/Joint/weights/best_ASC.pt')
model.load_state_dict(torch.load(pre_trained_model))

ModuleNotFoundError: No module named 'h5py'

In [4]:
len(lines_ASC)

NameError: name 'lines_ASC' is not defined

In [5]:
largs = {
        'trn_lines': trn_lines_ASC,
        'evl_lines': evl_lines_ASC,
        'd_label': d_label_ASC,
    }
trnset_gen_ASC, evlset_gen_ASC = get_loaders_ASC(largs, args)
trnset_gen_ASC_itr = cycle(trnset_gen_ASC)

NameError: name 'trn_lines_ASC' is not defined

In [1]:
import os
import torch
import h5py
import numpy as np
import soundfile as sf
import torchaudio as ta
 
from torch.utils import data
from utils import *


#####
# ASC
#####
def get_loaders_ASC(loader_args):
    trnset = Dataset_DCASE2020_t1(
        lines = loader_args['trn_lines'],
        base_dir = wav_file,
        d_label = loader_args['d_label'],
        verbose = verbose
    )
    trnset_gen = data.DataLoader(
        trnset,
        batch_size = bs_ASC,
        shuffle = True,
        num_workers = nb_worker,
        pin_memory = True,
        drop_last = True
    )

    evlset = Dataset_DCASE2020_t1(
        lines = loader_args['evl_lines'],
        trn = False,
        base_dir = wav_file,
        d_label = loader_args['d_label'],
        verbose = verbose
    )
    evlset_gen = data.DataLoader(
        evlset,
        batch_size = bs_ASC//3,
        shuffle = False,
        num_workers = nb_worker//2,
        pin_memory = True,
        drop_last = False
    )

    return trnset_gen, evlset_gen

class Dataset_DCASE2020_t1(data.Dataset):
    def __init__(self, lines, nb_frames = 0, trn = True, base_dir = '', d_label = '', verbose = 0):
        self.lines = lines 
        self.d_label = d_label
        self.base_dir = base_dir
        #self.nb_frames = nb_frames
        self.trn = trn
        self.verbose = verbose
        #self.return_label = return_label

        self.resample = ta.transforms.Resample(
            orig_freq=44100,
            new_freq=24000,
            resampling_method='sinc_interp_hann')
        self.melspec = ta.transforms.MelSpectrogram(
            24000,
            n_fft=2048,
            win_length=int(24000 * 0.001 * 40),
            hop_length=int(24000 * 0.001 * 20),
            window_fn=torch.hamming_window,
            n_mels=128)
        self.nb_samps = int(24000 * 0.001 * 20 * 250)   #(#samp_rate, into ms, #frames, #frames)
        self.margin = int(240000 - self.nb_samps)
        if not trn: self.TTA_mid_idx = int(self.nb_samps/2)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        k = self.lines[index]
        try:
            
            X, samp_rate = ta.load(os.path.join(self.base_dir,k), normalize = True)   #X.size : (1, 441000)
            #if self.verbose > 3: print('Loaded: ', X.size())norm
            X = self.resample(X) #X.size    : (1, 240000)
            #if self.verbose > 3: print('Resampled: ', X.size())
            #assert samp_rate == self.samp_rate
        except:
            raise ValueError('Unable to laod utt %s'%k)
        X = self._pre_emphasis(X)
        #if self.verbose > 3: print('Pre-emphasized: ', X.size())

        if self.trn:
            st_idx = np.random.randint(0, self.margin)
            X = X[:, st_idx:st_idx+self.nb_samps]
        else:
            l_X = []
            l_X.append(X[:,:self.nb_samps])
            l_X.append(X[:,self.TTA_mid_idx:self.TTA_mid_idx+self.nb_samps])
            l_X.append(X[:,-self.nb_samps:])
            X = torch.stack(l_X)
        X = self.melspec(X)

        ###### 
        X = torch.log(X)    #2020.8.2.

        #if self.verbose > 3: print('Mel-spec: ', X.size())
        X = self._utt_mvn(X)
        #print(X.size())   
        #trn: (1, 128, 251) dev: (3, 128, 251)

        #if self.trn:
        #    y = self.d_label[k.split('-')[0]]
        #    return X, y
        #else:
        #    return X
        y = self.d_label[k.split('-')[0]]
        return X, y

    
    def _pre_emphasis(self, x):
        return x[:,1:] - 0.97 * x[:, :-1] 

    def _utt_mvn(self, x):
        _m = x.mean(dim=-1, keepdim = True)
        _s = x.std(dim=-1, keepdim = True)
        _s[_s<0.001] = 0.001
        return (x - _m) / _s



ModuleNotFoundError: No module named 'h5py'